# "peanuts" key

This notebook generates the default key used on Linux when no keychain is available to store the password generated by Chrome, which is later derived to get the key used to encrypt the cookies.

In [5]:
:dep pbkdf2 = { version = "0.11.0", features = ["std", "sha1"] }

In [6]:
/// Salt for symmetric key derivation.
const SYMMETRIC_SALT: &[u8] = b"saltysalt";

/// Number of iterations to hash the password
/// which the key used to encrypt cookies is derived from.
const HASH_ROUNDS: u32 = 1;

/// Length of the derived key used by Chrome for AES-128.
const DERIVED_KEY_LENGTH: usize = 128;

use pbkdf2::{
    password_hash::{PasswordHasher, SaltString},
    Algorithm, Params, Pbkdf2,
};

/// Derives a key from a password using the same parameters as Chrome for
/// Linux platform.
fn derive_key_from_password<P: AsRef<[u8]>>(password: P) -> Vec<u8> {
    let salt = SaltString::b64_encode(SYMMETRIC_SALT).unwrap();

    let key = Pbkdf2.hash_password_customized(
        password.as_ref(),
        Some(Algorithm::Pbkdf2Sha1.ident()),
        None,
        Params {
            rounds: HASH_ROUNDS,
            output_length: DERIVED_KEY_LENGTH / 8,
        },
        &salt,
    ).unwrap();

    key.hash.unwrap().as_bytes().to_vec()
}

In [7]:
derive_key_from_password("peanuts")

[253, 98, 31, 229, 162, 180, 2, 83, 157, 250, 20, 124, 169, 39, 39, 120]